# Statistics notebook: Shared States
Accompanying the manuscript: Oosterwijk, Snoek, Rotteveel, Barrett, & Scholte. (submitted). "Shared states: Using MVPA to test neural overlap between self-focused emotion imagery and other-focused emotion understanding."

Lukas Snoek, February 2016 | lukassnoek@gmail.com <br>
https://github.com/lukassnoek/DecodingEmotions

### Code for main analysis
The code written for the main analysis (i.e. classification analysis of both the self-data and the generalization to the other-data) in contained in scripts hosted at the above Github repository, in the *main* folder.



In [1]:
from __future__ import division, print_function
import cPickle
import os
import glob
import os.path as op
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
import pandas as pd
import os
import os.path as op
import glob
from nilearn.plotting import plot_stat_map
from scipy.stats import f_oneway, ttest_rel, pearsonr, t
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

%matplotlib inline
sns.set_style("ticks")

plot_dir = '/home/lukas/Dropbox/PhD_projects/DecodingEmotions_SCAN/Plots'

The function below transforms 'raw' confusion matrics (in which the numbers indicate the number of trials assigned to a particular class, with the x-axis referring to the predicted class and the y-axis referring to the true class). The metrics precision (also known as Positive Predictive Value), recall (also known as the True Positive Rate), and F1-score (harmonic mean between precision and recall) are supported. In the article, precision-scores are reported.

In [4]:
def compute_score_confmat(confmat, metric='precision'):
    
    if metric == 'precision':
            return confmat / confmat.sum(axis=0)
    elif metric == 'recall':
        return confmat / confmat.sum(axis=1)
    elif metric == 'F1':
        precision = confmat / confmat.sum(axis=0)
        recall = confmat / confmat.sum(axis=1)
        return 2 * (recall * precision) / (recall + precision)               

# MAIN ARTICLE: STATISTICS & PLOTS
Below, the results from the validation set (as reported in the main article) are computed. The results from the optimization procedure and final analysis on the (final) optimization set are listed in section 2: SUPPLEMENTARY MATERIALS.

In [2]:
# Location of results from 100,000 iterations analysis
self_dir = '/media/lukas/data/DecodingEmotions/Validation_set/glm_zinnen/100000iter_results'
other_dir = '/media/lukas/data/DecodingEmotions/Validation_set/glm_HWW/100000iter_results'

## 1. Statistics of self-analysis
Below, we load in the subject-specific confusion matrices and convert the 'raw' metrics to more easily interpretable 'precision' scores:

$$ precision = \frac{true\ positives}{true\ positives + false\ positives} $$

The resulting ndarray thus contains precision-matrices for each subject, of which the average is plotted below.

In [5]:
self_files = glob.glob(op.join(self_dir, '*.pickle'))
s_confmat = np.zeros((len(self_files), 3, 3))
for i, f in enumerate(self_files):
    tmp = cPickle.load(open(f)).conf_mat
    s_confmat[i, :, :] = compute_score_confmat(tmp, metric='precision')

s_confmat[np.isnan(s_confmat)] = 0

In [ ]:
labels = ['Action', 'Interoception', 'Situation']
s_av_cm = s_confmat.mean(axis=0)
plt.figure(figsize=(8,6))
ax = sns.heatmap(s_av_cm, cmap=plt.cm.Greys, annot=True,
                 linewidths=0.1, linecolor='grey', vmax=.6, cbar=True, annot_kws={'fontsize': 20})
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=15)
plt.xticks(np.arange(3)+0.5, labels, fontsize=14)
plt.yticks(np.arange(3)+0.5, labels, fontsize=14)
plt.xlabel('Predicted class', fontsize=20, labelpad=19)
plt.ylabel('Actual class', fontsize=20, labelpad=19)
plt.title('Self-analysis', fontsize=25, y=1.03)
plt.tight_layout(pad=0.1)
plt.savefig(op.join(plot_dir, 'self_confmat.png'))

Below, we'll load in the confusion matrices from the permuted main analysis, which is exactly the same as the main analysis, yet with permuted class labels, yielding random models and corresponding confusion matrices.

In [7]:
# Loading in permutation confusion matrices
s_perm_dir = '/media/lukas/data/DecodingEmotions/Validation_set/glm_zinnen/permutation_results/'
s_dirs = glob.glob(op.join(s_perm_dir, 'perm_*'))
n = s_confmat.shape[0]

s_perm_mats = np.zeros((len(s_dirs), n, 3, 3))

for i, d in enumerate(s_dirs):
    mats = glob.glob(op.join(d, '*confmat.npy'))
    
    for ii, m in enumerate(mats):
        s_perm_mats[i, ii, :, :] = compute_score_confmat(np.load(m))

In [ ]:
# Significance: p-value = nr of perms above observed score / nr of total perms
# see: Nichols, T. E., & Holmes, A. P. (2002). Nonparametric permutation tests for functional neuroimaging: 
# a primer with examples. Human brain mapping, 15(1), 1-25
s_av_perm_mats = s_perm_mats.mean(axis=1)
s_p_cm = np.round((s_confmat.mean(axis=0) < s_av_perm_mats).sum(axis=0) / s_av_perm_mats.shape[0], 3)
print(s_p_cm)

## 2. Statistics of cross-analysis
Below, the same procedure as for the self-analysis is repeated.

In [ ]:
other_files = glob.glob(op.join(other_dir, '*.pickle'))
o_confmat = np.zeros((len(other_files), 3, 3))
for i, f in enumerate(other_files):
    tmp = cPickle.load(open(f)).conf_mat
    o_confmat[i, :, :] = compute_score_confmat(tmp, metric='precision')
o_confmat[np.isnan(o_confmat)] = 0

In [ ]:
o_av_cm = o_confmat.mean(axis=0)
plt.figure(figsize=(8,6))
ax = sns.heatmap(o_av_cm, cmap=plt.cm.Greys, annot=True,
                 linewidths=0.1, linecolor='grey', vmax=.6, cbar=True, annot_kws={'fontsize': 20})
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=15)
plt.xticks(np.arange(3)+0.5, labels, fontsize=14)
plt.yticks(np.arange(3)+0.5, labels, fontsize=14)
plt.xlabel('Predicted class', fontsize=20, labelpad=19)
plt.ylabel('Actual class', fontsize=20, labelpad=19)
plt.title('Cross-analysis', fontsize=25, y=1.03)
plt.tight_layout(pad=0.1)
plt.savefig(op.join(plot_dir, 'cross_confmat.png'))

In [ ]:
o_perm_dir = '/media/lukas/data/DecodingEmotions/Validation_set/glm_HWW/permutation_results/'
o_dirs = glob.glob(op.join(o_perm_dir, 'perm_*'))

# perm_mats shape = [nr perms, subjects, n_class, n_class]
o_perm_mats = np.zeros((len(o_dirs), o_confmat.shape[0], 3, 3))

for i, d in enumerate(o_dirs):
    mats = glob.glob(op.join(d, '*confmat.npy'))
    for ii, m in enumerate(mats):
        o_perm_mats[i, ii, :, :] = compute_score_confmat(np.load(m), 'precision')

In [ ]:
o_av_perm_mats = o_perm_mats.mean(axis=1)
o_p_cm = np.round((o_confmat.mean(axis=0) < o_av_perm_mats).sum(axis=0) / o_av_perm_mats.shape[0], 3)
print(o_p_cm)

## 3. Visualizing classification model in voxel-space
Here, the process of creating 'voxel plots' in which model parameters (i.e. linear svm weights) are calculated and backprojected. The weights, here, are the average weights across cross-validation iterations, which
were subsequently averaged across subjects. Ideally, these mean weight values are normalized by, for example, their standard error, yielding a standard t-value map. However, as the averaged *absolute* weights are biased to be higher than 0, we cannot do a t-test with H<sub>0 </sub>= 0. Therefore, we ran the main analysis again, but now with permuted labels, to keep track of the model weights yielded by random models (i.e. the permuted_scores variable). These permuted weights are subsequently averaged the same way as the 'true' weights. Given this information, we now calculate the t-value map as:

$$ t-value = \frac{(average\ true\ weight - average\ permuted\ weight)}{standard\ deviation\ true\ weight /(n-1)} $$


For more information, check the *skbold* package at Github:
https://github.com/lukassnoek/skbold/blob/master/skbold/utils/mvp_utils.py.
The code to keep track of svm weights and subsequently how these are averaged across iterations and subjects is contained in the classes *MvpResults* and *MvpAverageResults* respectively.

In [ ]:
vox_files = glob.glob('/media/lukas/data/DecodingEmotions/voxel_data/vox_results_mni/*.nii.gz')
perm_files = glob.glob('/media/lukas/data/DecodingEmotions/voxel_random/vox_results_mni/*.nii.gz')

vox_scores = np.zeros((len(vox_files), 91, 109, 91, 3))
perm_scores = np.zeros((len(perm_files), 91, 109, 91, 3))

for i, (v, p) in enumerate(zip(vox_files, perm_files)):
    perm_scores[i, :, :, :, :] = nib.load(p).get_data()
    vox_scores[i, :, :, :, :] = nib.load(v).get_data()

In [ ]:
import skbold.ROIs as roi # allows to import MNI152 brain
mni = op.join(op.dirname(roi.__file__), 'MNI152_2mm.nii.gz')
affine = nib.load(mni).get_affine()

In [ ]:
perm_average = np.abs(perm_scores).mean(axis=4)
vox_average = np.abs(vox_scores).mean(axis=4)
t_permuted = (vox_average.mean(axis=0) - perm_average.mean(axis=0)) / (vox_average.std(axis=0) / np.sqrt(n-1))
t_permuted[np.isnan(t_permuted)] = 0
t_permuted[np.isinf(t_permuted)] = 0
nib.save(nib.Nifti1Image(t_permuted, affine), op.join(plot_dir, 'modelweights'))

In [ ]:
img = nib.Nifti1Image(np.ma.array(t_permuted, mask=t_permuted < 0), affine)
weightsplot = plot_stat_map(img, display_mode='x', threshold=1.75, cut_coords=(-57, -50, -30, 0, 30, 50, 58),
                            vmax=5.5)
weightsplot.savefig(op.join(plot_dir, 'weightsplot'))

Below, a custom-function from the skbold-package is used to extract ROI-information about the weights-plot, i.e. how many voxels are contained 

In [ ]:
from skbold.postproc import extract_roi_info
print(roi.__file__)
df = extract_roi_info(op.join(plot_dir, 'modelweights.nii'), stat_threshold=1.75,
                 per_cluster=False, roi_type='bilateral')

**Note**: The tables\* describing, per brain region, the amount of voxels (and max. value, coordinates, etc.) included in the brain maps are computed using the *extract_roi_info()* function in the *skbold* package (module: misc_utils). Brain masks from the Harvard-Oxford Cortical atlas, which are probabilistic masks, were thresholded at 25% to minimize overlap between regions while covering as much as the brain as possible.

\* Supplementary Tables 3-6.

# SUPPLEMENTARY MATERIALS
Below, the statistics, plots, and other supporting analyses for the manuscript's supplementary materials are calculated/performed.

**Note**: Details about the pilot study in which the stimuli for the self-focused task were developed (supplementary table 1) can be send upon request. Also, no code is available accompanying Supplementary Figure 2 (and its corresponding analyses), because these results were acquired with an old analysis pipeline implemented in MATLAB (and plotted in R).

## Supplementary figure 1: percentage of trials executed successfully
Also lists some code for descriptive statistics mentioned in the main article (e.g. demographics data).

In [ ]:
# Some demographics
home = '/home/lukas/Dropbox/PhD_projects/DecodingEmotions_SCAN/Behavioral_data'
plot_dir = op.join(op.dirname(home), 'Plots')
demographics_file = op.join(home, 'demographics.csv')
demogr = pd.read_csv(demographics_file, sep=',')

m_age = demogr['age'].mean()
std_age = demogr['age'].std()
number_female = np.sum([g == 'F' for g in demogr['gender']])
print('Mean age: %f, std age: %f' % (m_age, std_age))
print('Proportion female: %f' % number_female)

In [ ]:
validation_file = op.join(home, 'validation_behav.csv')
val_data = pd.read_csv(validation_file, sep=',')

# val_data is in 'wide' format, but for seaborn-style plots, we need a long format
vars_to_melt = ['suc_other_act', 'suc_other_int', 'suc_other_sit', 'suc_self_act', 'suc_self_int', 'suc_self_sit']
df_long = pd.melt(val_data, id_vars=['sub_name'], value_vars=vars_to_melt)
df_long['condition'] = ['self' if 'self' in c else 'other' for c in df_long['variable']]

In [ ]:
ax1 = plt.subplot(121)
sns.barplot(x='variable', y='value', data=df_long[df_long['condition']=='self'], palette='Greys')
ax1.set_ylabel('% succesful', fontsize=15), ax1.set_xlabel('')
ax1.set_xticklabels(['Action', 'Interoception', 'Situation'], fontsize=10)
ax1.set_title('SF-task', fontsize=18)
ax2 = plt.subplot(122, sharex=ax1, sharey=ax1)
sns.barplot(x='variable', y='value', data=df_long[df_long['condition']=='other'], palette='Greys')
ax2.set_ylabel(''), ax2.set_xlabel('')
ax2.set_xticklabels(['Action', 'Interoception', 'Situation'], fontsize=10)
ax2.set_title('OF-task', fontsize=18)
plt.tight_layout()
sns.despine()
plt.savefig(op.join(plot_dir, 'success.png'), dpi=800)

### Accompanying inferential statistics
Including a one-way ANOVA for testing the overall effect and follow-up pairwise comparisons.

In [ ]:
n = val_data.shape[0]
k = 3 # i.e. 3 conditions (act, int, sit)
df1 = k - 1
df2 = n - k 

print('F-tests')
fval, pval = f_oneway(val_data['suc_self_act'], val_data['suc_self_int'], val_data['suc_self_sit'])
print('Self-data: F(%i, %i) = %f, p = %f' % (df1, df2, fval, pval))

fval, pval = f_oneway(val_data['suc_other_act'], val_data['suc_other_int'], val_data['suc_other_sit'])
print('Other-data: F(%i, %i) = %f, p = %f' % (df1, df2, fval, pval))

print('\nPairwise comparisons')
tval_si, pval_si = ttest_rel(val_data['suc_other_sit'], val_data['suc_other_int'])
print('Other sit-int: t(%i) = %f, p = %f)' % (n-1, tval_si, pval_si))

tval_sa, pval_sa = ttest_rel(val_data['suc_other_sit'], val_data['suc_other_act'])
print('Other sit-act: t(%i) = %f, p = %f)' % (n-1, tval_sa, pval_sa))

tval_ai, pval_ai = ttest_rel(val_data['suc_other_act'], val_data['suc_other_int'])
print('Other act-int: t(%i) = %f, p = %f)' % (n-1, tval_ai, pval_ai))
print('\nDescriptive statistics')

descr_stats = pd.DataFrame()
descr_stats['mean'] = val_data[['suc_other_act', 'suc_other_int', 'suc_other_sit']].mean()
descr_stats['se'] = val_data[['suc_other_act', 'suc_other_int', 'suc_other_sit']].std() / np.sqrt(n)
print(descr_stats)

## Supplementary figure 3: Confusion matrices from optimization-set
Is analyzed/computed/visualized the same way as the confusion matrices from the validation-set. Note, however, that no permutation analysis was performed, so the p-values corresponding the confusion matrix cells were calculated using a regular one-sample t-test (against H<sub>0</sub>: 0.333)

In [ ]:
# Location of results from 100,000 iterations analysis
optself_dir = '/media/lukas/data/DecodingEmotions/Optimization_set/glm_zinnen/analysis_results'
optother_dir = '/media/lukas/data/DecodingEmotions/Optimization_set/glm_HWW/analysis_results'

optself_files = glob.glob(op.join(optself_dir, '*.pickle'))
opts_confmat = np.zeros((len(optself_files), 3, 3))
for i, f in enumerate(optself_files):
    tmp = cPickle.load(open(f)).conf_mat
    opts_confmat[i, :, :] = compute_score_confmat(tmp, metric='precision')

opts_confmat[np.isnan(opts_confmat)] = 0

In [ ]:
labels = ['Action', 'Interoception', 'Situation']
opts_av_cm = opts_confmat.mean(axis=0)
plt.figure(figsize=(8,6))
ax = sns.heatmap(opts_av_cm, cmap=plt.cm.Greys, annot=True,
                 linewidths=0.1, linecolor='grey', vmax=.6, cbar=True, annot_kws={'fontsize': 20})
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=15)
plt.xticks(np.arange(3)+0.5, labels, fontsize=14)
plt.yticks(np.arange(3)+0.5, labels, fontsize=14)
plt.xlabel('Predicted class', fontsize=20, labelpad=19)
plt.ylabel('Actual class', fontsize=20, labelpad=19)
plt.title('Self-analysis', fontsize=25, y=1.03)
plt.tight_layout(pad=0.1)
plt.savefig(op.join(plot_dir, 'optself_confmat.png'))

In [ ]:
n = opts_confmat.shape[0]
tmat_self = (opts_confmat.mean(axis=0) - 0.333) / (opts_confmat.std(axis=0) / np.sqrt(n-1))
pmat_self = np.array([stats.t.sf(np.abs(tt), n-1)*2 for tt in tmat_self])
print(pmat_self.round(4))

In [ ]:
optother_files = glob.glob(op.join(optother_dir, '*.pickle'))
opto_confmat = np.zeros((len(optother_files), 3, 3))
for i, f in enumerate(optother_files):
    tmp = cPickle.load(open(f)).conf_mat
    opto_confmat[i, :, :] = compute_score_confmat(tmp, metric='precision')

opto_confmat[np.isnan(opto_confmat)] = 0

In [ ]:
labels = ['Action', 'Interoception', 'Situation']
opto_av_cm = opto_confmat.mean(axis=0)
plt.figure(figsize=(8,6))
ax = sns.heatmap(opto_av_cm, cmap=plt.cm.Greys, annot=True,
                 linewidths=0.1, linecolor='grey', vmax=.6, cbar=True, annot_kws={'fontsize': 20})
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=15)
plt.xticks(np.arange(3)+0.5, labels, fontsize=14)
plt.yticks(np.arange(3)+0.5, labels, fontsize=14)
plt.xlabel('Predicted class', fontsize=20, labelpad=19)
plt.ylabel('Actual class', fontsize=20, labelpad=19)
plt.title('Cross-analysis', fontsize=25, y=1.03)
plt.tight_layout(pad=0.1)
plt.savefig(op.join(plot_dir, 'optother_confmat.png'))

In [ ]:
n = opto_confmat.shape[0]
tmat_other = (opto_confmat.mean(axis=0) - 0.333) / (opto_confmat.std(axis=0) / np.sqrt(n-1))
pmat_other = np.array([stats.t.sf(np.abs(tt), n-1)*2 for tt in tmat_other])
print(pmat_other.round(3))

## Supplementary figure 4: relation self- and cross-scores & their distribution 
The plot below was included to show that, while precision scores between the self- and cross-analyses are uncorrelated, the distributions of scores from both analyses seem quite different. More specifically, the self-scores appear normally distributed, while the cross-scores appear to be have two 'peaks' in their distribution. In other words, there might be two groups of subjects: one which is capable of generalization from self to other and one group which is not (around chance level), which seems to be irrespective of their classification performance on the self-focused task. 

In [ ]:
s_prec, c_prec = val_data.self_precision, val_data.cross_precision

gs = GridSpec(3, 2)

plt.figure(figsize=(6, 6))
sns.set_style('ticks')
plt.subplot(gs[0:2, 0:2])

sns.regplot(s_prec, c_prec, color='red')
r, p = pearsonr(s_prec, c_prec)
plt.text(0.32, 0.6, 'r = %.2f, p = %.3f' % (r, p), fontsize=17)
plt.xlabel('Precision self-analysis', fontsize=15)
plt.ylabel('Precision cross-analysis', fontsize=15)

plt.subplot(gs[2, 0])
sns.distplot(s_prec, bins=6)
plt.xlabel('Precision self-analysis')
plt.ylabel('Frequency')
plt.xticks(np.arange(0.2, 1.2, 0.2))
plt.yticks(np.arange(0, 6, 1))

plt.subplot(gs[2, 1])
sns.distplot(c_prec, bins=6, color='green')
plt.xlabel('Precision cross-analysis')
plt.xticks(np.arange(0.2, 1.2, 0.2))

sns.despine()
plt.tight_layout()#pad=3)
plt.savefig(op.join(plot_dir, 'self_cross_relation.png'), dpi=600)

## Supplementary figure 5: precision vs. recall

In [ ]:
prec_self = np.zeros((len(self_files), 3, 3))
recall_self = np.zeros((len(self_files), 3, 3))

for i, f in enumerate(self_files):
    tmp = cPickle.load(open(f)).conf_mat
    prec_self[i, :, :] = compute_score_confmat(tmp, metric='precision')
    recall_self[i, :, :] = compute_score_confmat(tmp, metric='recall')

prec_self[np.isnan(prec_self)] = 0
recall_self[np.isnan(recall_self)] = 0
prec_self, recall_self = prec_self.mean(axis=0), recall_self.mean(axis=0)

prec_other = np.zeros((len(other_files), 3, 3))
recall_other = np.zeros((len(other_files), 3, 3))

for i, f in enumerate(other_files):
    tmp = cPickle.load(open(f)).conf_mat
    prec_other[i, :, :] = compute_score_confmat(tmp, metric='precision')
    recall_other[i, :, :] = compute_score_confmat(tmp, metric='recall')

prec_other[np.isnan(prec_other)] = 0
recall_other[np.isnan(recall_other)] = 0
prec_other, recall_other = prec_other.mean(axis=0), recall_other.mean(axis=0)

In [ ]:
sns.set_style('white')
plt.figure(figsize=(8,7))
plt.subplots_adjust(hspace=0.3, wspace=0.1)

labels2 = ['A', 'I', 'S']
gs2 = GridSpec(2, 2)

plt.subplot(gs2[0, 0])
sns.heatmap(prec_self, cmap=plt.cm.Greys, annot=True, cbar=False,
            linewidths=0.1, linecolor='grey', vmax=.6, annot_kws={'fontsize': 20}, vmin=0.05)
plt.xticks(np.arange(3)+0.5, [''] * len(labels2))
plt.yticks(np.arange(3)+0.5, labels, fontsize=11)
plt.ylabel('Actual class', fontsize=18, labelpad=19)
#plt.title('Precision self-analysis', fontsize=18, y=1.05)

plt.subplot(gs2[0, 1])
sns.heatmap(prec_other, cmap=plt.cm.Greys, annot=True, cbar=False,
            linewidths=0.1, linecolor='grey', vmax=.6, annot_kws={'fontsize': 20}, vmin=0.05)
plt.xticks(np.arange(3)+0.5, [''] * len(labels2))
plt.yticks(np.arange(3)+0.5, [''] * len(labels2))
#plt.title('Precision cross-analysis', fontsize=18, y=1.05)

plt.subplot(gs2[1, 0])
sns.heatmap(recall_self, cmap=plt.cm.Greys, annot=True, cbar=False,
            linewidths=0.1, linecolor='grey', vmax=.6, annot_kws={'fontsize': 20}, vmin=0.05)
plt.xticks(np.arange(3)+0.5, labels, fontsize=11)
plt.yticks(np.arange(3)+0.5, labels, fontsize=11)
plt.xlabel('Predicted class', fontsize=18, labelpad=19)
plt.ylabel('Actual class', fontsize=18, labelpad=19)
#plt.title('Recall self-analysis', fontsize=18, y=1.05)

plt.subplot(gs2[1, 1])
sns.heatmap(recall_other, cmap=plt.cm.Greys, annot=True, cbar=False,
            linewidths=0.1, linecolor='grey', vmax=.6, annot_kws={'fontsize': 20}, vmin=0.05)
plt.xticks(np.arange(3)+0.5, labels, fontsize=11)
plt.yticks(np.arange(3)+0.5, [''] * len(labels2))
plt.xlabel('Predicted class', fontsize=18, labelpad=19)
#plt.title('Recall cross-analysis', fontsize=18, y=1.05)

plt.savefig(op.join(plot_dir, 'precision_recall_plots.png'), dpi=600)

In [ ]:
import glob
import os
import warnings
import os.path as op
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import label_binarize
from joblib import Parallel, delayed
from skbold.utils.mvp_utils import (MvpResults, MvpAverageResults,
                                         DataHandler)
from skbold.transformers.transformers import *
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score, confusion_matrix

In [ ]:
project_dir = '/media/lukas/data/DecodingEmotions/Validation_set'
self_dir = op.join(project_dir, 'glm_zinnen')
other_dir = op.join(project_dir, 'glm_HWW')
self_paths = glob.glob(op.join(self_dir, 'sub*'))
other_paths = glob.glob(op.join(other_dir, 'sub*'))

# Parameters
zvalue = 2.3

scaler = StandardScaler()
transformer = MeanEuclidean(cutoff=zvalue, normalize=True)
clf = SVC(kernel='linear', probability=True, decision_function_shape='ovr')
pipeline = Pipeline([('transformer', transformer),
                     ('scaler', scaler),
                     ('classifier', clf)])
probas = np.zeros((len(self_paths), 90, 3))

for i, (self_path, other_path) in enumerate(zip(self_paths, other_paths)):
    self_data = DataHandler(identifier='merged', shape='2D').load_separate_sub(self_path)
    other_data = DataHandler(identifier='', shape='2D').load_separate_sub(other_path)

    y_ovr = label_binarize(other_data.y, classes=[0, 1, 2])
    pipeline.fit(self_data.X, self_data.y)
    probas[i, :, :] = pipeline.predict_proba(other_data.X)
    #print(confusion_matrix(other_data.y, pipeline.predict(other_data.X)))
av_probas = probas.mean(axis=0)

In [ ]:
# Credits to scikit-learn:
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
sns.set_style('white')
precision = dict()
recall = dict()
average_precision = dict()
n_classes = 3
y_test = y_ovr
y_score = av_probas
labels = ['Action', 'Interoception', 'Situation']
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                        y_score[:, i])
    average_precision[i] = average_precision_score(y_test[:, i], y_score[:, i])

for i in range(n_classes):
    plt.plot(recall[i], precision[i],
             label='{0} (area = {1:0.2f})'
                   ''.format(labels[i], average_precision[i]))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall', fontsize=15)
plt.ylabel('Precision', fontsize=15)
plt.title('Precision-recall curves for cross-analysis', fontsize=15, y=1.03)
plt.legend(loc="lower right", fontsize=12)
#plt.show()
plt.savefig(op.join(plot_dir, 'pr_curve.png'), dpi=600)

In [ ]:
from sklearn.metrics import precision_score, recall_score

pr, rc = np.zeros((50, 3)), np.zeros((50, 3))
indices = [np.arange(30), np.arange(30, 60), np.arange(60, 90)]

for i,t in enumerate(np.arange(0, 1, 1/50)):
    for ii in range(3):
        
        rc[i, ii] = (av_probas[indices[ii], ii] > t).mean()
        total_pred = (av_probas[:, ii] > t).sum()
        if total_pred == 0:
            pr[i, ii] = (av_probas[indices[ii], ii] > t).sum() / av_probas.shape[1]
        else:
            pr[i, ii] = (av_probas[indices[ii], ii] > t).sum() / (av_probas[:, ii] > t).sum()
        
rc[np.isnan(rc)] = 0
pr[np.isnan(pr)] = 0